# Plot EMA and Candlestick

## Strategy

In [1]:
import talib.abstract as ta
import pandas_ta as pdta

from lettrade import DataFeed, Strategy, indicator as i
from lettrade.exchange.backtest import ForexBackTestAccount, let_backtest


class SmaCross(Strategy):
    ema1_window = 9
    ema2_window = 21

    def indicators(self, df: DataFeed):
        df["ema1"] = ta.EMA(df, timeperiod=self.ema1_window)
        df["ema2"] = ta.EMA(df, timeperiod=self.ema2_window)

        df["signal_ema_crossover"] = i.crossover(df.ema1, df.ema2)
        df["signal_ema_crossunder"] = i.crossunder(df.ema1, df.ema2)

        df["cdl_doji"] = pdta.cdl_doji(df.open, df.high, df.low, df.close)

    def next(self, df: DataFeed):
        if len(self.orders) > 0 or len(self.positions) > 0:
            return

        if df.l.signal_ema_crossover[-1]:
            price = df.l.close[-1]
            self.buy(size=0.1, sl=price - 0.001, tp=price + 0.001)
        elif df.l.signal_ema_crossunder[-1]:
            price = df.l.close[-1]
            self.sell(size=0.1, sl=price + 0.001, tp=price - 0.001)

    def plot(self, df: DataFeed) -> dict:
        from lettrade.plot import PlotColor
        from lettrade.plot.plotly import (
            plot_candlestick,
            plot_line,
            plot_merge,
        )

        plot_ema1 = plot_line(df["ema1"], color="yellow")
        plot_ema2 = plot_line(df["ema2"], color="green")
        plot_cdl_doji = plot_candlestick(
            df[df["cdl_doji"] != 0],
            name=f"Doji {df.name}",
            width=3,
        )
        return plot_merge(
            plot_ema1,
            plot_ema2,
            plot_cdl_doji,
        )


lt = let_backtest(
    strategy=SmaCross,
    datas="example/data/data/EURUSD_5m-0_1000.csv",
    account=ForexBackTestAccount,
    # plotter=None,
)

## Run

In [2]:
lt.run()

# Strategy                <class '__main__.SmaCross'>
Start                       2024-05-13 21:15:00+00:00
End                         2024-05-17 08:30:00+00:00
Duration                              3 days 11:15:00
Start Balance                                 10000.0
Equity [$]                                    9975.32
Equity Peak [$]                              10002.22
PL [$]                                         -24.68
PL [%]                                          -0.25
Buy & Hold PL [%]                                0.63
Max. Drawdown [%]                               -0.46
Avg. Drawdown [%]                               -0.12
Max. Drawdown Duration                2 days 05:30:00
Avg. Drawdown Duration                0 days 12:30:00
                                                     
# Positions                                        15
Win Rate [%]                                      0.4
Fee [$]                                         -0.58
Best Trade [%]              

## Plot

In [6]:
lt.plot(layout=dict(height=2_000))